<a href="https://colab.research.google.com/github/Imteaz1998/Genrative-AI/blob/main/9_Pinecone.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pypdf
!pip install langchain
!pip install pinecone-client

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 300.7/300.7 kB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.8/244.8 kB 9.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.4/85.4 kB 6.5 MB/s eta 0:00:00


In [2]:
!pip install langchain-huggingface
!pip install tiktoken
%pip install --upgrade --quiet  langchain-google-genai
%pip install -qU langchain_community pypdf pillow

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 56.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 49.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 33.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 11.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 73.6 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjitlin

# Import all the required libraries

In [3]:
from langchain_community.document_loaders import PyPDFDirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_google_genai import GoogleGenerativeAI, HarmBlockThreshold, HarmCategory
from langchain.vectorstores import Pinecone
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate
import os

**Load The PDF FIles**

In [4]:
!mkdir pdfs

In [5]:
!gdown 1hPQlXrX8FbaYaLypxTmeVOFNitbBMlEE -O pdfs/yolov7paper.pdf
!gdown 1vILwiv6nS2wI3chxNabMgry3qnV67TxM -O pdfs/rachelgreecv.pdf

Downloading...
From: https://drive.google.com/uc?id=1hPQlXrX8FbaYaLypxTmeVOFNitbBMlEE
To: /content/pdfs/yolov7paper.pdf
100% 2.27M/2.27M [00:00<00:00, 36.3MB/s]
Downloading...
From: https://drive.google.com/uc?id=1vILwiv6nS2wI3chxNabMgry3qnV67TxM
To: /content/pdfs/rachelgreecv.pdf
100% 271k/271k [00:00<00:00, 85.6MB/s]


In [6]:
loader = PyPDFDirectoryLoader("pdfs")
data = loader.load()

# Splitting the Extracted data

In [7]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=20)

In [8]:
text_chunks = text_splitter.split_documents(data)

In [9]:
text_chunks

[Document(metadata={'producer': 'pdfTeX-1.40.21', 'creator': 'LaTeX with hyperref', 'creationdate': '2022-07-07T00:21:22+00:00', 'author': '', 'keywords': '', 'moddate': '2022-07-07T00:21:22+00:00', 'ptex.fullbanner': 'This is pdfTeX, Version 3.14159265-2.6-1.40.21 (TeX Live 2020) kpathsea version 6.3.2', 'subject': '', 'title': '', 'trapped': '/False', 'source': 'pdfs/yolov7paper.pdf', 'total_pages': 15, 'page': 0, 'page_label': '1'}, page_content='YOLOv7: Trainable bag-of-freebies sets new state-of-the-art for real-time object\ndetectors\nChien-Yao Wang1, Alexey Bochkovskiy, and Hong-Yuan Mark Liao1\n1Institute of Information Science, Academia Sinica, Taiwan\nkinyiu@iis.sinica.edu.tw, alexeyab84@gmail.com, and liao@iis.sinica.edu.tw\nAbstract\nYOLOv7 surpasses all known object detectors in both\nspeed and accuracy in the range from 5 FPS to 160 FPS\nand has the highest accuracy 56.8% AP among all known'),
 Document(metadata={'producer': 'pdfTeX-1.40.21', 'creator': 'LaTeX with hyperr

In [10]:
len(text_chunks)

168

In [24]:
import os

os.environ['Gemini_API_KEY'] = "API"

# Embedding

In [12]:
embeddings=HuggingFaceEmbeddings()

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

1_Pooling%2Fconfig.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [13]:
result = embeddings.embed_query("How are you akdjlakjd jadkaldjakl akjdlajdla!")

In [14]:
len(result)

768

# Initializing Pinecone

In [ ]:
!pip install pinecone-client==2.2.1 langchain==0.0.235


In [22]:
PINECONE_API_KEY = os.environ.get('PINECONE_API_KEY', 'API')
PINECONE_API_ENV = os.environ.get('PINECONE_API_ENV', 'Serverless')

In [28]:
from pinecone import Pinecone

pc = Pinecone(api_key=PINECONE_API_KEY)
index = pc.Index("test")


# Create Embeddings from chunk and creating indexes in pinecoin

In [33]:
os.environ['PINECONE_API_KEY'] = 'API'

index_name = "test"

In [38]:
from langchain.vectorstores import Pinecone
docsearch = Pinecone.from_texts([t.page_content for t in text_chunks], embeddings, index_name=index_name)

# If you already have vector database you can do

In [39]:
docsearch = Pinecone.from_existing_index(index_name, embeddings)
docsearch

# Similarity Search

In [40]:
query = "YOLOv7 outperforms which models"

In [41]:
docs = docsearch.similarity_search(query, k=3)

In [42]:
docs

[Document(metadata={}, page_content='tion, in terms of the amount of parameters and computation,\nYOLOv7-X reduces 22% of parameters and 8% of compu-\ntation compared to YOLOv5-X (r6.1), but improves AP by\n2.2%.\n7'),
 Document(metadata={}, page_content='tion, in terms of the amount of parameters and computation,\nYOLOv7-X reduces 22% of parameters and 8% of compu-\ntation compared to YOLOv5-X (r6.1), but improves AP by\n2.2%.\n7'),
 Document(metadata={}, page_content='over, we train YOLOv7 only on MS COCO dataset from\nscratch without using any other datasets or pre-trained\nweights.\n10')]

## Creating a LLM Model Wrapper

In [43]:
llm = GoogleGenerativeAI(
    model="gemini-pro",
    google_api_key="API",
    safety_settings={
        HarmCategory.HARM_CATEGORY_DANGEROUS_CONTENT: HarmBlockThreshold.BLOCK_NONE,
    },
)

In [44]:
qa = RetrievalQA.from_chain_type(llm=llm, chain_type="stuff", retriever=docsearch.as_retriever())

# Q/A

In [45]:
query = "YOLOv7 outperforms which models"

In [46]:
qa.run(query)

<ipython-input-46-fbc3ad05536a>:1: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  qa.run(query)


"I don't know"

In [47]:
query = "Rachel Green Experience"

In [48]:
qa.run(query)

"I don't know."

In [49]:
import sys


In [50]:
while True:
  user_input = input(f"Input Prompt: ")
  if user_input == 'exit':
    print('Exiting')
    sys.exit()
  if user_input == '':
    continue
  result = qa({'query': user_input})
  print(f"Answer: {result['result']}")

Input Prompt: what is yolo v7


<ipython-input-50-c213a85929f6>:8: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  result = qa({'query': user_input})


Answer: I don't know.
Input Prompt: what you know?
Answer: The provided context is about professional service and publications. It does not mention anything about what I know, so I do not know the answer to this question.
Input Prompt: yolo
Answer: I don't know.


KeyboardInterrupt: Interrupted by user